### Definição dos KPIs

**Objetivo**: Avaliar o impacto de uma estratégia de cupons na retenção de usuários 

<!-- KPIs:
1. Taxa de conversão: Dos usuários que receberam a oferta vs. os que não receberam, qual a porcentagem que efetivamente realizou um pedido durante o período da campanha?

2. Frequência de Pedidos: Em média, os usuários do grupo de teste fizeram mais pedidos que os do grupo de controle?

3. Ticket Médio: O valor médio por pedido foi maior no grupo de teste?

4. Taxa de Retenção (Engajamento e recompra, dado que só ficaram pedidos de 2019): Dos clientes que fizeram pelo menos um pedido em 2019, qual a porcentagem que voltou para fazer um segundo pedido (ou mais)? -->

| Título do KPI                     | O que é?                                                                 | Como é calculado                                   | Nome da Coluna  |
| --------------------------------- | ------------------------------------------------------------------------ | -------------------------------------------------- | ------------------------ |
| **Número de pedidos por usuário** | Total de pedidos que um cliente fez no período do teste.                 | `countDistinct(order_id)` por `customer_id`        | `orders_per_user`        |
| **Valor total gasto por usuário** | Quanto o cliente gastou no total (com capping aplicado).                 | `sum(order_total_amount_capped)` por `customer_id` | `total_spent_per_user`   |
| **Ticket médio por pedido**       | Valor médio gasto por pedido (quanto o cliente gasta em cada pedido).    | `total_spent_per_user / orders_per_user`           | `avg_ticket_per_user`    |
| **Quantidade total de itens**     | Soma da quantidade de itens pedidos por um usuário.                      | `sum(total_items_quantity)` por `customer_id`      | `total_items`            |
| **Itens por pedido (média)**      | Quantidade média de itens por pedido do usuário.                         | `total_items / orders_per_user`                    | `avg_items_per_order`    |
| **Desconto médio por pedido**     | Valor médio de desconto aplicado em cada pedido.                         | `sum(total_items_discount) / orders_per_user`      | `avg_discount_per_order` |
| **Retenção durante o período**    | Se o cliente fez mais de um pedido durante o período do teste (1 = sim). | `when(orders_per_user >= 2, 1).otherwise(0)`       | `is_retained`            |

In [0]:
import pyspark.sql.functions as f
from constants import SILVER_LAYER_PATH, GOLD_LAYER_PATH, USER_METRICS_GOLD_COLUMN_COMMENTS

from scipy.stats import ttest_ind, chi2, mannwhitneyu
from statsmodels.stats.proportion import proportions_ztest

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
abt_df = spark.table(f"{SILVER_LAYER_PATH}.abt_final")

In [0]:
def add_column_comments(table_name, comments_dict):
    for col_name, comment_text in comments_dict.items():
        safe_comment = comment_text.replace('"', '\\"')
        
        sql_command = f"""
        ALTER TABLE {table_name} ALTER COLUMN {col_name} COMMENT "{safe_comment}"
        """
        spark.sql(sql_command)

In [0]:
user_metrics_df = (abt_df.groupBy("customer_id", "is_target") 
    .agg(
        f.countDistinct("order_id").alias("orders_per_user"),
        f.sum("order_total_amount_capped").alias("total_spent_per_user"),
        f.sum("total_items_quantity").alias("total_items"),
        f.sum("total_items_discount").alias("total_discount"),
        f.min("days_since_signup").alias("days_since_signup")  # caso tenha várias linhas por cliente
    ) 
    .withColumn("avg_ticket_per_user", f.col("total_spent_per_user") / f.col("orders_per_user")) 
    .withColumn("avg_items_per_order", f.col("total_items") / f.col("orders_per_user")) 
    .withColumn("avg_discount_per_order", f.col("total_discount") / f.col("orders_per_user")) 
    .withColumn("is_retained", f.when(f.col("orders_per_user") >= 2, 1).otherwise(0)))

In [0]:
gold_user_metrics_table_name = f"{GOLD_LAYER_PATH}.user_metrics"

spark.sql(f"DROP TABLE IF EXISTS {gold_user_metrics_table_name}")

(user_metrics_df.write
 .partitionBy("is_target")
 .format("delta")
 .mode("overwrite")
 .saveAsTable(gold_user_metrics_table_name))

# adicionando descrição das colunas
add_column_comments(gold_user_metrics_table_name, USER_METRICS_GOLD_COLUMN_COMMENTS)

print("Tabela 'user_metrics' da camada Gold criada com sucesso!")

In [0]:
group_stats = user_metrics_df.groupBy("is_target").agg(
    f.count("*").alias("n_users"),
    f.avg("orders_per_user").alias("avg_orders"),
    f.avg("total_spent_per_user").alias("avg_total_spent"),
    f.avg("avg_ticket_per_user").alias("avg_ticket"),
    f.avg("avg_items_per_order").alias("avg_items"),
    f.avg("avg_discount_per_order").alias("avg_discount"),
    f.avg("is_retained").alias("retention_rate")
)

display(group_stats)

In [0]:
def test_avg_ticket_consistency():
    df_test = user_metrics_df.withColumn(
        "recalculated_avg_ticket", f.col("total_spent_per_user") / f.col("orders_per_user")
    )
    inconsistent = df_test.filter(
        f.abs(f.col("avg_ticket_per_user") - f.col("recalculated_avg_ticket")) > 0.01
    )
    assert inconsistent.count() == 0, "Inconsistência no cálculo do avg_ticket_per_user"
    
def test_control_target_presence():
    groups = user_metrics_df.select("is_target").distinct().collect()
    values = {row["is_target"] for row in groups}
    assert values == {True, False}, f"Grupos faltando: {values}"

def test_no_null_kpis():
    nulls = user_metrics_df.select(
        *[f.sum(f.col(c).isNull().cast("int")).alias(c) for c in [
            "orders_per_user", "total_spent_per_user", "avg_ticket_per_user"
        ]]
    ).collect()[0].asDict()
    
    assert all(v == 0 for v in nulls.values()), f"Nulls detectados em KPIs: {nulls}"

def test_retention_logic():
    errors = user_metrics_df.filter(
        ((f.col("orders_per_user") >= 2) & (f.col("is_retained") != 1)) |
        ((f.col("orders_per_user") < 2) & (f.col("is_retained") != 0))
    )
    assert errors.count() == 0, "Inconsistência na lógica de is_retained"

In [0]:
test_avg_ticket_consistency()
test_control_target_presence()
test_no_null_kpis()
test_retention_logic()

In [0]:
user_pd = user_metrics_df.toPandas()

control_group = user_pd[user_pd["is_target"] == False]
test_group = user_pd[user_pd["is_target"] == True]

kpis = [
    "orders_per_user",
    "total_spent_per_user",
    "avg_ticket_per_user",
    "avg_items_per_order",
    "avg_discount_per_order",
    "days_since_signup"
]

for column in kpis:
    t_stat, p_val = ttest_ind(test_group[column], control_group[column], equal_var=False)
    u_stat, p_val_mw_test = mannwhitneyu(test_group[column], control_group[column], alternative='two-sided')
    print(f"{column}: p-valor (T-test) = {p_val:.5f}, p-valor (Mann-Whitney) = {p_val_mw_test:.5f}")

In [0]:
kpis = [
    "orders_per_user",
    "total_spent_per_user",
    "avg_ticket_per_user",
    "avg_items_per_order",
    "avg_discount_per_order",
    "days_since_signup"
]

results = []

for kpi in kpis:
    mean_t = test_group[kpi].mean()
    mean_c = control_group[kpi].mean()
    diff = mean_t - mean_c
    pct_diff = (diff / mean_c) * 100

    if pd.notnull(mean_c) and mean_c != 0:
        pct_diff = (diff / mean_c) * 100
    else:
        pct_diff = np.nan
    
    t_stat, p_val = ttest_ind(test_group[kpi], control_group[kpi], equal_var=False)
    results.append({
        "kpi": kpi,
        "mean_control": mean_c,
        "mean_target": mean_t,
        "diff": diff,
        "pct_diff": pct_diff,
        "p_value": p_val
    })

df_stats = pd.DataFrame(results)

In [0]:
df_stats["significance"] = df_stats["p_value"] < 0.05

# Mapa de calor com a diferença percentual
plt.figure(figsize=(10, 5))
heat_data = df_stats.pivot_table(index="kpi", values="pct_diff")
sns.heatmap(heat_data, annot=True, fmt=".2f", cmap="RdYlGn", center=0, cbar_kws={"label": "% de diferença"})

plt.title("Diferença percentual entre grupos (Target - Controle)")
plt.ylabel("KPI")
plt.tight_layout()
plt.show()

print("\nSignificância estatística dos KPIs (p < 0.05):")
print(df_stats[["kpi", "pct_diff", "p_value", "significance"]])

In [0]:
plt.figure(figsize=(12, 6))
sns.histplot(data=user_pd, x='orders_per_user', hue='is_target', kde=True, bins=50)
plt.title('Distribuição do Total de pedidos por Usuário')
plt.show()

In [0]:
plt.figure(figsize=(12, 6))
sns.histplot(data=user_pd, x='total_spent_per_user', hue='is_target', kde=True, bins=50)
plt.title('Distribuição do Gasto Total por Usuário')
plt.show()

**Conclusão**
KPIs com impacto estatístico significativo:

`orders_per_user`: cupom incentivou maior frequência de pedidos.

`total_spent_per_user`: cupom aumentou o gasto total por cliente.

KPIs sem impacto estatístico:

`avg_ticket_per_user`, `avg_items_per_order`: os pedidos não ficaram maiores ou mais caros, só mais frequentes.

`avg_discount_per_order`: provavelmente não houve registro de desconto por pedido na variável.

`days_since_signup`: não houve desequilíbrio relevante entre os grupos em relação ao tempo de cadastro (bom sinal).

In [0]:
ret_target = test_group["is_retained"].sum()
ret_control = control_group["is_retained"].sum()
n_target = len(test_group)
n_control = len(control_group)

count = np.array([ret_target, ret_control])
nobs = np.array([n_target, n_control])

stat, pval = proportions_ztest(count, nobs)
print(f"is_retained → Z = {stat:.2f}, p = {pval:.5f}")

O Z-score mede quantos desvios padrão acima da média esperada está a diferença entre as proporções dos dois grupos. Um valor de Z > 2 já seria considerado estatisticamente significativo.

* `Z = 89` é absurdamente alto: a diferença entre os grupos é enorme e consistente.

O p-valor é a probabilidade de observar uma diferença tão grande por acaso, assumindo que não há diferença real (hipótese nula).

* `P ≈ 0` indica que a diferença observada não é aleatória.

Lembrete dos dados de `is_retained`:

Retenção no grupo controle: **47,6%**

Retenção no grupo target: **57,6%**

Diferença absoluta: **+10 pontos percentuais**

Base de usuários: **~800 mil no total**

#### Resultado geral dos KPIs

O impacto são significativos em: `orders_per_user`, `total_spent_per_user` e `is_reteined`.

A campanha de cupons teve impacto estatisticamente significativo na frequência e no gasto total por cliente, mas não alterou o valor médio dos pedidos nem o volume de itens por compra. Isso sugere que os cupons aumentaram o número de compras realizadas, sem influenciar o tamanho ou valor de cada uma. O grupo de controle e o grupo alvo estavam balanceados em termos de maturidade (antiguidade na base), o que reforça a confiabilidade do teste.

O resultado reforça que a campanha teve **EFEITO POSITIVO na retenção e no uso contínuo da plataforma**.


| KPI                      | Diferença % Target vs Controle | p-value  | Significativo?   | Interpretação                                                                                |
| ------------------------ | ------------------------------ | -------- | ---------------- | -------------------------------------------------------------------------------------------- |
| `orders_per_user`        | **+13,33%**                    | 0.000000 | ✅ Sim            | **Grupo com cupom fez mais pedidos por usuário** — fortemente significativo.                 |
| `total_spent_per_user`   | **+13,35%**                    | 0.000000 | ✅ Sim            | **Usuários com cupom gastaram mais no total** — altamente significativo.                     |
| `avg_ticket_per_user`    | +0,04%                         | 0.703    | ❌ Não            | O valor **médio por pedido** praticamente não mudou com o cupom.                             |
| `avg_items_per_order`    | +0,30%                         | 0.261    | ❌ Não            | **Quantidade média de itens por pedido** não sofreu alteração relevante.                     |
| `avg_discount_per_order` | *NaN*                          | *NaN*    | ❌ Não (inválido) | Dados ausentes ou inválidos (provavelmente sempre zero) — impossível testar.                 |
| `days_since_signup`      | +0,51%                         | *NaN*    | ❌ Não (inválido) | Valores provavelmente ausentes ou sem variabilidade relevante — análise inconclusiva.        |
| `is_retained`            | **+10,0 pontos percentuais.**                 | 0.000000 | ✅ Sim            | **Cupom aumentou a proporção de usuários com mais de um pedido** — fortemente significativo. |

### Análise de viabilidade financeira

**KPIs**:
| Indicador               | Controle   | Target     | Diferença (%) |
| ----------------------- | ---------- | ---------- | ------------- |
| Gasto médio por usuário | R\$ 126,95 | R\$ 143,89 | **+13,35%**   |
| Usuários no grupo       | 360.541    | 445.924    | —             |
| Retenção (>1 pedido)    | 47,6%      | 57,6%      | **+10 p.p.**  |

**Premissas**
| Premissa                                                                   | Valor                             |
| -------------------------------------------------------------------------- | --------------------------------- |
| Valor médio do cupom distribuído                                           | R\$ 10,00                         |
| Todos os usuários do grupo target **receberam e usaram** o cupom           | Sim (pior cenário – custo máximo) |
| O aumento no gasto total por usuário (R\$ 16,94) é **decorrente do cupom** | Sim                               |
| Consideramos apenas o **efeito direto no período da campanha**             | Sim                               |


In [0]:
n_target = test_group.shape[0]
coupon_value = 10.00
total_cost = n_target * coupon_value

earn_per_user = df_stats.loc[df_stats["kpi"] == "total_spent_per_user", "diff"].values[0]
total_return = earn_per_user * n_target

print(f"Custo total dos cupons: R${total_cost:.2f}")
print(f"Lucro por usuário: R$ {earn_per_user:.2f}")
print(f"Retorno total: R$ {total_return:.2f}")

In [0]:
net_profit = total_return - total_cost
roi = (net_profit / total_cost) * 100

print(f"Lucro líquido: R$ {net_profit:,.2f}")
print(f"Retorno sobre o investimento (ROI): {roi:.2f}%")

A campanha gerou mais dinheiro do que custou, com um ROI de quase `70%`, o que significa que `para cada R$ 1 investido, retornou R$ 1,69`.
Isso, por si só, já caracteriza a campanha como financeiramente viável.